In [1]:
let leftA = Left "error" :: Either String Int
:type leftA

let rightB = Right 4 :: Either String Int
:type rightB

double :: Num a => a -> a
double n =
  n * 2
:type double

leftA :: Either String Int

rightB :: Either String Int

double :: forall a. Num a => a -> a

We have two `Either String Int` types and a function that operates outside of any context. What if we want to double a number inside the `Either` type?

In [2]:
:type fmap
fmap double leftA
fmap double rightB
double <$> rightB  -- Infix convenience for `fmap`.

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

Left "error"

Right 8

Right 8

There are two ways to look at `fmap`.

1. Take a function that operates outside of a context, apply it inside a context to produce a new value inside a context.
2. Take a function that operates outside of a context, and “lift” it into a context. Viewing it this way, the type signature becomes `fmap :: Functor f => (a -> b) -> (f a -> f b)`.

Note that the `Functor` instance of `fmap` is written such that the `Left` case is always preserved, it represents an “error”. The `Right` case is the “correct” case and these entries will be mapped over.

Now what if we had two numbers in an `Either` context, and we wanted to add them together? We cannot do this with `fmap`.

In [3]:
let rightC = Right 6 :: Either String Int
:type rightC
:type (+)

rightC :: Either String Int

(+) :: forall a. Num a => a -> a -> a

What we need is the `Applicative` instance of `Either`. Specifically, the `apply` operator `<*>`. We can also look at the `pure` function, which lifts a value into a context.

In [4]:
:type (<*>)
:type pure
:type pure (+)
:type pure (+) <*> rightB

(<*>) :: forall (f :: * -> *) a b. Applicative f => f (a -> b) -> f a -> f b

pure :: forall (f :: * -> *) a. Applicative f => a -> f a

pure (+) :: forall (f :: * -> *) a. (Num a, Applicative f) => f (a -> a -> a)

pure (+) <*> rightB :: Either String (Int -> Int)

Notice how we now have the partially applied `(+)` function in context.

In [5]:
pure (+) <*> rightB <*> rightC

Right 10

As it turns out, there is already a convenience method for this!

In [6]:
import Control.Applicative
:type liftA2
liftA2 (+) rightB rightC

liftA2 :: forall a b c (f :: * -> *). Applicative f => (a -> b -> c) -> f a -> f b -> f c

Right 10

In [7]:
(.+) = liftA2 (+)
rightB .+ rightC
leftA .+ rightB

Right 10

Left "error"

Monad time!

In [8]:
import Control.Monad
:type return
:type (>>=)

return :: forall (m :: * -> *) a. Monad m => a -> m a

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

So this takes a contextual value, applies a function, and returns the value after application within the context. This function is *not* lifted into a context (unlike the one we use with `<*>`) and does not use a contextual value, it does return one though.

`return` is just like `pure`. It lifts a value into context.

In [9]:
triple :: (Num a, Monad m) => a -> m a
triple n =
  return $ n * 3

:type triple
:type (rightB >>=)  -- Same thing as next expression, infix!
:type (>>=) rightB

triple :: forall a (m :: * -> *). (Monad m, Num a) => a -> m a

(rightB >>=) :: forall b. (Int -> Either String b) -> Either String b

(>>=) rightB :: forall b. (Int -> Either String b) -> Either String b

We now have a function that is just waiting to receive a function that operates on the `Int` value (`Int -> Either String b`) and will then return the final context (`Either String b`).

In [10]:
rightB >>= triple
:type rightB >>= triple
rightB >>= triple >>= triple

Right 12

rightB >>= triple :: Either String Int

Right 36